### Edgar Moises Hernandez-Gonzalez
#### Asesores: Pilar Gomez-Gil, Erik Bojorges-Valdez
#### Instituto Nacional de Astrofísica Óptica y Electrónica (INAOE)
#### 25/11/20-03/12/20
#### Tesis: Clasificación de señales EEG basada en representaciones bidimensionales y redes neuronales convolucionales
#### Clasificacion de EEG con CNN-2D o CNN-2D + LSTM
##### Caracteristicas = Espectrogramas STFT o Escalogramas CWT

##### Busqueda de cuadricula

In [ ]:
# activar google drive
# no es necesario si se ejecuta local
from google.colab import drive

In [ ]:
# montar drive
# no es necesario si se ejecuta local
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ejecutar desde aqui si se ejecuta en su computadora local, importar librerias
import numpy as np
import pandas as pd
import time
from scipy.signal import spectrogram
import pywt
import cv2
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import (Conv2D, MaxPool2D, Flatten, Dense, Dropout,
                          TimeDistributed, LSTM)
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K

In [ ]:
# para que este script funcione debe de decir 'channels_last'
# si dice 'channels_first' no sirve
K.image_data_format()

'channels_last'

In [ ]:
# leer .csv con los datos de train y test
# se debe especificar la ruta
# header = None significa que los .csv no tienen encabezado
x_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2b/Datos/MI-EEG-B9T.csv",
                      header=None)
x_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2b/Datos/MI-EEG-B9E.csv",
                     header=None)
y_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2b/Datos/etiquetas_train_9.csv",
                      header=None)
y_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2b/Datos/etiquetas_test_9.csv",
                     header=None)

In [ ]:
# imprimir la forma de las matrices de datos
# x_train y x_test son matrices donde las filas son el numero de ejemplos
# y las columnas son el numero de segundos por la frecuencia de muestreo
# por el numero de canales
# y_train y y_test son las etiquetas
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(400, 3000)
(320, 3000)
(400, 1)
(320, 1)


In [ ]:
# unir train y test, despues se hara la division con cross-validation
# x (matrices) lo unimos con pandas (mas facil para el calculo de STFT o CWT)
# y (etiquetas) lo unimos con numpy
x = pd.concat([x_train, x_test])
y = np.concatenate((y_train, y_test))

In [ ]:
# imprimir la forma de las matrices de datos
print(x.shape)
print(y.shape)

(720, 3000)
(720, 1)


In [ ]:
# calcular el numero de muestras
# en caso de no tener el y_test se puede utilizar el x_test
n_samples = len(y)

print("n_samples:", n_samples)

n_samples: 720


In [ ]:
# calcular el numero de clases
# esto se podria calcular asi n_clases = len(np.unique(y))
# la y puede ser train o test
n_classes = len(np.unique(y))

print("n_classes:", n_classes)

n_classes: 2


In [ ]:
# calcular espectrogramas con STFT
# concatenacion vertical de los espectrogramas de los n canales
# unir_espectrogramas_vertical(matriz de x_train o x_test,
# frecuencia de muestreo, alto, ancho, n_canales, tamano del segmento de senal,
# puntos superpuestos para calcular la STFT)
def unir_espectrogramas_vertical(data, fs, alto, ancho, n_canales, pts_sig,
                                 pts_superpuestos):
  #fs = fs #frecuencia de muestreo
  # crear matriz 3D para almacenar todas las imagenes de los STFT
  datos = np.zeros((data.shape[0],alto, ancho))
  
  # crear matriz 2D donde se guardara cada imagen del STFT
  temporal = np.zeros((alto, ancho))

  for i in range(data.shape[0]): # n muestras
    for j in range(n_canales): # n canales
      # puntos_sig = duracion en segundos de la señal x frecuencia de muestreo
      # para una señal de 2 seg con fs de 250 Hz, puntos_sig = 500
      sig = data.iloc[i, j*pts_sig:(j+1)*pts_sig]
      
      # espectrograma con STFT
        # senal,
        # fs=frecuencia de muestreo,
        # window='tipo de ventana',
        # nperseg=tamano de ventana,
        # noverlap=puntos superpuestos,
        # nfft=tamano de la FFT con relleno de ceros, Si=None la longitud de FFT es nperseg
        # scaling=‘density’: power spectral density, ‘spectrum’: power spectrum
      f, t, Sxx = spectrogram(sig, fs=fs, window='hann', nperseg=fs,
                              noverlap=pts_superpuestos, nfft=fs*2,
                              scaling='spectrum')
      
      # concatenacion vertical de canales
      # espectrograma genera 45 filas (frecuencias de 8 a 30Hz con pasos de
      # 0.5Hz) y 31 columnas (tiempo)
      # las frecuncias de 8 a 30 Hz estan entre los indices de 16 a 60
      temporal[j*45:(j+1)*45, :] = Sxx[16:61, :]

    datos[i] = temporal
    if i % 100 == 0: # esto es para ver como avanza
      print(i)
  return datos

In [ ]:
# calcular escalogramas con CWT
# concatenacion vertical de los escalogramas de los n canales
# unir_escalogramas_vertical(matriz de x_train o x_test, frecuencia de muestreo,
# alto, ancho, n_canales, tamano del segmento de senal):
def unir_escalogramas_vertical(data, fs, alto, ancho, n_canales, pts_sig):
  # ancho y alto, para el resize usando Open CV
  dim = (int(np.floor(ancho/2)), int(np.floor(alto/2))) # ancho, alto
  
  # calcular escalas para Wavelet compleja de Morlet 3-3
  # con frecuencias de 8 a 30 Hz con pasos de 0.5 Hz
  escalas = pywt.scale2frequency('cmor3-3', np.arange(8,30.5,0.5)) / (1/fs)
  
  # crear matriz 3D para almacenar todas las imagenes
  datos = np.zeros((data.shape[0], int(np.floor(alto/2)),
                    int(np.floor(ancho/2))))
  
  # crear matriz 2D donde se guardara cada imagen
  temporal = np.zeros((alto, ancho))

  for i in range(data.shape[0]): # n muestras
    for j in range(n_canales): # n canales
      # puntos_sig = duracion en segundos de la señal x frecuencia de muestreo
      # para una señal de 2 seg con fs de 250 Hz, puntos_sig = 500
      sig = data.iloc[i, j*pts_sig:(j+1)*pts_sig]
      
      # escalograma con CWT
        # senal
        # escalas
        # nombre de la wavelet, en este caso compleja de Morlet 3-3
        # periodo de muestreo = 1 / frecuencia de muestreo
      coef, freqs = pywt.cwt(sig, escalas, 'cmor3-3',
                             sampling_period = (1 / fs))
      
      # concatenacion vertical de canales
      # espectrograma genera 45 filas (frecuencias de 8 a 30Hz con pasos de
      # 0.5Hz) y 31 columnas (tiempo)
      # dado que cmor3-3 genera numeros complejos, calcular el modulo
      temporal[j*45:(j+1)*45, :] = abs(coef)

    # resize usando una interpolacion interarea con OpenCV
    resized = cv2.resize(temporal, dim, interpolation=cv2.INTER_AREA)
    datos[i] = resized
    if i % 100 == 0: # esto solo es para ver como avanza
      print(i)
  return datos

In [ ]:
# seleccionar STFT o CWT
inicio = time.time()

# STFT, llamar a unir_espectrogramas_vertical(data, fs, alto, ancho,
# n_canales, pts_sig, pts_superpuestos))
# descomentar la siguiente linea para STFT
x = unir_espectrogramas_vertical(x, 250, 135, 31, 3, 1000, 225)

# CWT, llamar a unir_escalogramas_vertical(data, fs, alto, ancho,
# n_canales, pts_sig)
# descomentar la siguiente linea para CWT
#x = unir_escalogramas_vertical(x, 250, 135, 1000, 3, 1000)

fin = time.time()
print("Tiempo:", fin - inicio)

0
100
200
300
400
500
600
700
Tiempo: 1.525439739227295


In [ ]:
# forma de los espectrogramas (STFT) o escalogramas (CWT)
# la forma debe ser tridimensional
# la primera dimension es igual al numero de ejemplos
# la segunda dimension es lo alto de las imagenes calculadas con STFT o CWT
# la tercera dimension es lo ancho de las imagenes calculadas con STFT o CWT
print(x.shape)

(720, 135, 31)


In [ ]:
# imprimir el maximo, minimo y la media del x (imagenes de STFT o CWT)
print(np.max(x))
print(np.min(x))
print(np.mean(x))

10.7888912416627
3.407406192918792e-08
0.06609588538579113


In [ ]:
# convertir a float
x = x.astype('float32')

# escalar los valores en un rango de 0 a 1 (normalizar)
# dividir entre el max redondeado al techo
x /= np.ceil(np.max(x))

In [ ]:
# imprimir el maximo, minimo y la media del x (imagenes de STFT o CWT)
# despues de normalizar los datos
print(np.max(x))
print(np.min(x))
print(np.mean(x))

0.98080826
3.0976421e-09
0.006008719


In [ ]:
# seleccionar reshape a 4D (para CNN-2D) o 5D (para CNN-2D + LSTM)
# para CNN-2D usar a 4D, para CNN-2D + LSTM usar a 5D

# convertir de 3D a 4D (CNN-2D)
# descomentar la siguiente linea para usar CNN-2D
x = x.reshape((x.shape[0], x.shape[1], x.shape[2], 1))

# convertir de 3D a 5D (CNN-2D + LSTM)
# descomentar la siguiente linea para usar CNN-2D + LSTM
#x = x.reshape((x.shape[0], 1, x.shape[1], x.shape[2], 1))

# imprimir la forma
print(x.shape)
print(x.shape)

(720, 135, 31, 1)
(720, 135, 31, 1)


In [ ]:
# crear red neuronal CNN-2D
# se debe especificar el numero de filtros, el tamano de los filtros
# y el numero de neuronas en la capa oculta del MLP (Dense)
# modelo secuencial de dos capas convolucionales 2D con activacion relu y
# relleno de ceros y max pooling de 2x2
# aplanar los mapas de caracteristicas para convertirlas de una matriz 3D a
# un vector 1D
# MLP de una capa oculta con relu
# regularizacion con dropout de 0.5
# la ultima capa utiliza una funcion softmax para obtener una distribucion
# de probabilidad
# optimizador Adam con tasa de aprendizaje de 1x10-4
# la funcion de perdida es la entropia cruzada categorica
# (se utiliza sparse para no convertir las etiquetas a one hot)
# la metrica a evaluar es el accuracy
def CNN_2D(n_filtros, tamano_filtros, n_neuronas):
  model = Sequential()
  model.add(Conv2D(n_filtros, tamano_filtros, activation='relu', padding='same',
                   input_shape=x_train.shape[1:]))
  model.add(MaxPool2D((2,2)))
  model.add(Conv2D(n_filtros, tamano_filtros, activation='relu', padding='same'))
  model.add(MaxPool2D((2,2)))
  model.add(Flatten())
  model.add(Dense(n_neuronas, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(n_classes, activation='softmax'))

  optimizer = Adam(lr=1e-4)
  model.compile(optimizer = optimizer,
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  return model

In [ ]:
# crear red neuronal CNN-2D + LSTM
# se debe especificar el numero de filtros, el tamano de los filtros,
# el numero de neuronas en la capa oculta del MLP (Dense) y el numero de
# unidades LSTM 
# modelo secuencial de dos capas convolucionales 2D con activacion relu y
# relleno de ceros y max pooling de 2x2
# aplanar los mapas de caracteristicas para convertirlas de una matriz 3D a
# un vector 1D
# red recurrente LSTM con activacion tangente hiperbolica y dropout de 0.5
# (esta va entre las capas convolucionales y el MLP)
# MLP de una capa oculta con relu
# regularizacion con dropout de 0.5
# la ultima capa utiliza una funcion softmax para obtener una distribucion
# de probabilidad
# optimizador Adam con tasa de aprendizaje de 1x10-4
# la funcion de perdida es la entropia cruzada categorica
# (se utiliza sparse para no convertir las etiquetas a one hot)
# la metrica a evaluar es el accuracy
# para poder conectar CNN-2D con LSTM se debe utilizar la capa TimeDistributed
def CNN_2D_LSTM_TD(n_filtros, tamano_filtros, n_neuronas, unidades_LSTM):
  model = Sequential()
  model.add(TimeDistributed(Conv2D(n_filtros, tamano_filtros, activation='relu',
                                   padding='same'),
                            input_shape=x_train.shape[1:]))
  model.add(TimeDistributed(MaxPool2D((2,2))))
  model.add(TimeDistributed(Conv2D(n_filtros, tamano_filtros, activation='relu',
                                   padding='same')))
  model.add(TimeDistributed(MaxPool2D((2,2))))
  model.add(TimeDistributed(Flatten()))
  model.add(LSTM(unidades_LSTM, activation='tanh', dropout=0.5))
  model.add(Dense(n_neuronas, activation='relu'))
  model.add(Dense(n_classes, activation='softmax'))

  optimizer = Adam(lr=1e-4)
  model.compile(optimizer = optimizer,
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  return model

In [ ]:
# clasificador scikit-learn para Keras
# build_fn = modelo
# para STFT = 400 epochs, para CWT = 100 epochs
# batch_size: su tamano es opcional (depende del dataset)
clf = KerasClassifier(build_fn = CNN_2D, epochs=400, batch_size=36, verbose=0)
#clf = KerasClassifier(build_fn = CNN_2D_LSTM_TD, epochs=400, batch_size=36, verbose=0)

In [ ]:
# definir todos los posibles valores para los hiperparametros
# descomentar unidades LSTM cuando se utilize CNN_2D_LSTM_TD
n_filtros = [2, 4]
tamano_filtros = [(3,3), (15,3)]
n_neuronas = [16, 32]
#unidades_LSTM = [4, 8]
hiperparametros = dict(n_filtros=n_filtros,
                       tamano_filtros=tamano_filtros,
                       n_neuronas=n_neuronas) #,unidades_LSTM=unidades_LSTM)

In [ ]:
# busqueda de cuadricula
# GridSearchCV(clasificador scikit-learn, hiperparametros,
# fold cross-validation)
grid = GridSearchCV(clf, hiperparametros, cv=2)

In [ ]:
inicio = time.time()

# iniciar la busqueda de cuadricula
grid.fit(x, y)

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 153.83689618110657


In [ ]:
# parametros con los que se obtuvo el mejor accuracy usando cross-validation
print(grid.best_params_)
print(grid.best_score_)

{'n_filtros': 4, 'n_neuronas': 32, 'tamano_filtros': (3, 3)}
0.7486110925674438


In [ ]:
# accuracy para cada combinacion de hiperparametros
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))

0.732 (+/-0.119) for {'n_filtros': 2, 'n_neuronas': 16, 'tamano_filtros': (3, 3)}
0.744 (+/-0.122) for {'n_filtros': 2, 'n_neuronas': 16, 'tamano_filtros': (15, 3)}
0.736 (+/-0.133) for {'n_filtros': 2, 'n_neuronas': 32, 'tamano_filtros': (3, 3)}
0.722 (+/-0.094) for {'n_filtros': 2, 'n_neuronas': 32, 'tamano_filtros': (15, 3)}
0.739 (+/-0.094) for {'n_filtros': 4, 'n_neuronas': 16, 'tamano_filtros': (3, 3)}
0.744 (+/-0.139) for {'n_filtros': 4, 'n_neuronas': 16, 'tamano_filtros': (15, 3)}
0.749 (+/-0.142) for {'n_filtros': 4, 'n_neuronas': 32, 'tamano_filtros': (3, 3)}
0.719 (+/-0.089) for {'n_filtros': 4, 'n_neuronas': 32, 'tamano_filtros': (15, 3)}


In [ ]:
# informacion de la GPU
# solo utilizar si esta utilizando usa GPU de Google Colab
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu Nov 26 04:36:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    52W /  70W |    871MiB / 15079MiB |     42%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------